# 유형1 : 데이터 전처리 작업 3문제 * 10 [30점]

### 1. 데이터 불러오기

In [ ]:
import pandas as pd

# csv 파일 불러오기 
df = pd.read_csv('파일경로.csv')
## 불러오면서 인덱스 컬럼 설정
df_csv1 = pd.read_csv('파일경로.csv', index_col='인덱스로 삼을 칼럼명')
## 불러오면서 구분자 지정
df_csv2 = pd.read_csv('파일경로.csv', sep='\t')
## 불러올 때 언어 설정
df_csv3 = pd.read_csv('파일경로.csv', encoding='euc-kr')  # 한글지원 : euc-kr, cp949

# excel 파일 불러오기 (엑셀은 엔진 설정해주기!!)
df_excel = pd.read_excel('파일경로.xlsx', engine='openpyxl')

### 2. 데이터 구조 확인 

In [ ]:
# 처음부터 n개 행의 데이터 확인
df.head()
# 끝부터 n개 행의 데이터 확인
df.tail()

# 데이터프레임 row개수, colum개수, Not null,dtype 등 정보 확인
df.info()
# 데이터프레임 통계 정보 확인
df.describe()

# 데이터프레임의 행, 열의 수를 (행, 열)형태의 튜블로 반환
df.shape
# 데이터프레임의 (행 x 열)의 전체 데이터 수를 반환
df.size

#데이터프레임의 인덱스 확인 (보통 0으로 시작해서 몇으로 끝나고 스텝이 몇인지 등을 보여줌)
df.index
# 데이터프레임의 컬럼 확인
df.columns          # 타입 index
df.columns.values   # 타입 array
# 데이터프레임의 구성요소 2차원으로 보기
df.values           # 타입 array



### 3. 데이터 타입 변경

### 4. 데이터 정렬

### 5. 데이터 통계 확인

### 6. 시각화

### 7. 데이터 정제하기

#### 1) 결측치

#### 2) 이상치

### 3) 파생 변수

유형2 : 데이터 전처리 및 모델링 1문제(40*1)
    1. 데이터 불러오기 : pd.read_csv('.csv', index_col=0)
    2. 데이터 정제하기
        1) 결측치 제거 or 대체
        2) 이상치 제거 or 대체
    3. 데이터 변환하기
        1) 불리언 인덱싱
        2) np.where(조건식)
    4. 범주형 데이터 라벨링
        1) LabelEncoding
        2) one-hot-encoding(getdummys()?)
    5. 뉴메리컬 데이터 스케일링
        1) StandardScaler
        2) MinMaxScaler
    6. 모델링
        1) 지도학습 분류 : RandomforestClassifier
        2) 지도학습 회귀 : RandomforestRegressor
        3) 비지도학습 군집화
    7. 모델 적용
        1) train_test_split(층화 추출 옵션stratify = y)
        2) new - fit - predict
    8. 모델 성능 확인 : 문제에서 요구하는 평가지표에 맞게 수행
        1) 지도 학습
            - 정확도(accuracy)
            - 정밀도(precision)
            - 재현율(recall)
            - 특이도(Specificity)
            - f1_score : 
            - ROC(Receiver Operating Characteristic) 곡선: 
            - AUC(Are Under Curve) : 
        2) 회귀 학습
            - SSE(Sum Squared Error) : 
            - MSE(Mean Squared Error) : 
            - RMSE(Root Mean Squared Error) :
            - MAE(Mean Absolute Error) : 
            - 결정계수(R2) :
            - Adjusted R2 : 
            - MSPE(Mean Squared Percentage Error) : 
            - MAPE(Mean Absolute Percentage Error) :
            - RMSLE(Root Mean Squared Logarithmic Error) : 
            - AIC(Akaike Information Criterion) : 
            - BIC(Bayes Information Criteria) : 
        3) 비지도 학습
            - 실루엣 계수(Silhouette) : 
            - Dunn lndex : 
            - 군집 분석
            - 연관 분석
    9. 예측 값 저장 
        submission.columns([''])
        to_csv('.csv', index=False)



유형3 : 데이터 검증 2문제(15*2)
    1. 데이터 불러오기 : pd.read_csv('.csv', index_col=0)
    2. 데이터 파악하기 : data.describe()
    3. 데이터 검증하기
    귀무가설(h0) 대신 대립가설(h1)을 선택할 시 오류를 범할 최대의 확률 값으로 0.05 이하이면 유의하여 대립가설 채택, 일반적으로 채택/기각은 귀무가설 기준
        01) t-검정
            (1)
            (2)
            (3)
        02) 정규성 검정
            (1)
            (2)
        03) One-way ANOVA
        04) Two-way ANOVA
        05) 카이제곱 검정
            (1) 독립성 검정
            (2) 동질성 검정
            (3) 적합성 검정
        06) 순위 검정
            (1) 윌콕슨 순위합 검정
            (2) 크루스칼 - 왈리스 검정
        07) 부호검정
            (1) 부호검정
            (2) 맥네마 검정
        08) 다변량 추론
            (1) 다중 회귀 분석

    4. 결과값 입력하기



t-검정 이해하기
